In [1]:
from keras import datasets

(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data(path='mnist.npz')

Using TensorFlow backend.


In [2]:
# kearas 支持channel first和channel last
# channel first: [batch, channel, height, width]
# channel last: [batch, height, width, channel]  默认
# 可以在./keras/keras.json 进行配置

In [13]:
from keras import backend as K

img_rows, img_cols = 28, 28

if K.image_data_format() ==  'channel_first':
    x_train  = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols) # 1为通道数
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train  = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1) # 1为通道数
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [14]:
X_train = x_train.astype('float32')
X_test = x_test.astype('float32')
# 数据归一化
X_train /= 255
X_test /= 255

In [15]:
from keras.utils import np_utils

n_classes = 10
print("one-hot encoding befor:", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
print("one-hot encoding after:", Y_train.shape)
Y_test = np_utils.to_categorical(y_test, n_classes)

one-hot encoding befor: (60000,)
one-hot encoding after: (60000, 10)


In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()

## feature extraction
# 第一层卷积， 32个3X3卷积核，激活函数relu
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) # input_shape [28, 28, 1]

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
# 查看模型结构

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [20]:
for layer in model.layers:
    print(layer.get_output_at(0).get_shape().as_list())


[None, 26, 26, 32]
[None, 24, 24, 64]
[None, 12, 12, 64]
[None, 12, 12, 64]
[None, None]
[None, 128]
[None, 128]
[None, 10]


In [21]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [22]:
history = model.fit(X_train, Y_train, batch_size=128, epochs=5, verbose=2, validation_data=(X_test, Y_test))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
 - 108s - loss: 0.2252 - acc: 0.9319 - val_loss: 0.0488 - val_acc: 0.9833
Epoch 2/5
 - 106s - loss: 0.0815 - acc: 0.9757 - val_loss: 0.0371 - val_acc: 0.9868
Epoch 3/5
 - 114s - loss: 0.0592 - acc: 0.9820 - val_loss: 0.0291 - val_acc: 0.9897
Epoch 4/5
 - 107s - loss: 0.0509 - acc: 0.9847 - val_loss: 0.0308 - val_acc: 0.9893
Epoch 5/5
 - 106s - loss: 0.0424 - acc: 0.9869 - val_loss: 0.0303 - val_acc: 0.9894


In [23]:
# 保存模型
import os
import tensorflow.gfile as gfile

save_dir = './mnist/model/'

if gfile.Exists(save_dir):
    gfile.DeleteRecursively(save_dir)
gfile.MakeDirs(save_dir)

model_name = 'keras_mnist.h5'
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('model saved as path {}'.format(model_path))

model saved as path ./mnist/model/keras_mnist.h5


In [24]:
# 加载模型
from keras.models import load_model
import numpy as np

mnist_model = load_model(model_path)

# 统计模型在测试集的分类结果

loss_and_metrics = mnist_model.evaluate(X_test, Y_test, verbose=2)

print("Test Loss: {}".format(loss_and_metrics[0]))
print("Test Accuracy: {}%".format(loss_and_metrics[1]*100))

predicted_classes = mnist_model.predict_classes(X_test)

correct_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]

print("Classified correctly count: {}".format(len(correct_indices)))
print("Classified incorrectly count: {}".format(len(incorrect_indices)))

Test Loss: 0.030295624294431764
Test Accuracy: 98.94%
Classified correctly count: 9894
Classified incorrectly count: 106
